In [280]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,LabelEncoder,RobustScaler,TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier,XGBRegressor
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier

In [281]:
df=pd.read_csv(r'C:\Users\pavan\Desktop\data science\decision tree\ensemble methods\BOOSTING_METHODS\uae_ecom_fraud_100k.csv')

In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 36 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   transaction_id          100000 non-null  int64  
 1   user_id                 100000 non-null  int64  
 2   timestamp_utc           100000 non-null  object 
 3   amount_aed              100000 non-null  float64
 4   currency                100000 non-null  object 
 5   payment_method          100000 non-null  object 
 6   device_type             100000 non-null  object 
 7   browser                 100000 non-null  object 
 8   merchant_category       100000 non-null  object 
 9   items_count             100000 non-null  int64  
 10  avg_item_price          100000 non-null  float64
 11  shipping_city           100000 non-null  object 
 12  billing_city            100000 non-null  object 
 13  shipping_billing_match  100000 non-null  int64  
 14  ip_address           

In [283]:
df.head()

,transaction_id,user_id,timestamp_utc,amount_aed,currency,payment_method,device_type,browser,merchant_category,items_count,...,local_hour,odd_hour,is_fraud,fraud_flag_ip,fraud_flag_mismatch,fraud_flag_velocity,fraud_flag_new_account,fraud_flag_prev_cb,fraud_flag_odd_hour,data_source
0,500086758,1000466,2024-01-01T00:02:03Z,71.47,AED,card,desktop,Chrome,Pharmacy,3,...,4,1,0,0,0,0,0,0,1,synthetic:ecom-fraud-uae-v1
1,500060464,1002828,2024-01-01T00:03:11Z,70.25,AED,card,mobile,Chrome,Groceries,1,...,4,1,0,0,0,0,1,0,1,synthetic:ecom-fraud-uae-v1
2,500039909,1010349,2024-01-01T00:04:10Z,116.25,AED,bank_transfer,desktop,Chrome,Beauty,2,...,4,1,0,0,0,0,0,0,1,synthetic:ecom-fraud-uae-v1
3,500074412,1015360,2024-01-01T00:05:14Z,99.41,AED,apple_pay,mobile,Safari,Toys,2,...,4,1,0,0,0,0,0,0,1,synthetic:ecom-fraud-uae-v1
4,500038110,1022253,2024-01-01T00:11:03Z,529.40,AED,card,desktop,Safari,Travel,2,...,4,1,0,0,0,0,1,1,1,synthetic:ecom-fraud-uae-v1


In [284]:
df=df.drop(columns=['transaction_id','user_id','timestamp_utc','data_source','ip_address'])

In [285]:
cat_col=df.select_dtypes(exclude='number').columns
df[cat_col] = df[cat_col].astype('category')

num_col=df.select_dtypes(include='number')

In [286]:
for col in cat_col:
    print(col,'-->',df[col].value_counts(),df[col].nunique())

currency --> currency
AED    100000
Name: count, dtype: int64 1
payment_method --> payment_method
card             70006
paypal            8027
apple_pay         7999
google_pay        7858
bank_transfer     6110
Name: count, dtype: int64 5
device_type --> device_type
mobile     50002
desktop    45063
tablet      4935
Name: count, dtype: int64 3
browser --> browser
Chrome              49530
Safari              25126
Firefox             12147
Edge                 7028
Samsung Internet     4091
Other                2078
Name: count, dtype: int64 6
merchant_category --> merchant_category
Fashion             18009
Electronics         17944
Groceries           15080
Digital Goods       12054
Home & Furniture     7973
Beauty               6966
Pharmacy             6031
Sports               6016
Travel               5013
Toys                 4914
Name: count, dtype: int64 10
shipping_city --> shipping_city
Dubai             25053
Abu Dhabi         18089
Mumbai            11978
Sharjah        

In [287]:
numerical=df[['amount_aed','avg_item_price','ip_risk_score','card_age_days','user_account_age_days','local_hour']]

In [288]:
numerical

,amount_aed,avg_item_price,ip_risk_score,card_age_days,user_account_age_days,local_hour
0,71.47,23.82,35.3,85,365,4
1,70.25,70.25,30.2,123,-192,4
2,116.25,58.12,32.7,542,1744,4
3,99.41,49.70,30.3,981,102,4
4,529.40,264.70,18.5,515,-59,4
...,...,...,...,...,...,...
99995,28.37,7.09,70.5,345,-294,3
99996,19.85,6.62,57.2,955,864,3
99997,65.16,32.58,41.0,457,35,3
99998,87.11,21.78,54.7,368,971,3


In [289]:
for col in num_col:
    print(col,'-->',df[col].value_counts(),df[col].nunique())

amount_aed --> amount_aed
33.99      20
24.19      19
17.90      18
23.18      18
24.98      17
           ..
465.19      1
172.88      1
650.54      1
140.19      1
2610.74     1
Name: count, Length: 33713, dtype: int64 33713
items_count --> items_count
2    35816
1    30479
3    21661
4     8689
5     2572
6      643
7      120
8       14
9        6
Name: count, dtype: int64 9
avg_item_price --> avg_item_price
13.96     33
19.86     33
18.58     31
15.12     31
12.84     31
          ..
260.73     1
681.50     1
242.89     1
740.63     1
233.79     1
Name: count, Length: 25205, dtype: int64 25205
shipping_billing_match --> shipping_billing_match
1    92665
0     7335
Name: count, dtype: int64 2
ip_risk_score --> ip_risk_score
0.0     340
40.1    273
39.5    268
33.3    260
37.1    260
       ... 
92.1      1
97.4      1
94.5      1
86.5      1
97.7      1
Name: count, Length: 998, dtype: int64 998
card_present --> card_present
0    96970
1     3030
Name: count, dtype: int64 2
card_ag

In [290]:
x=df.drop(columns='is_fraud')
y=df['is_fraud']

In [291]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

In [292]:
num=numerical.columns
cat=x_train.select_dtypes(exclude='number').columns

In [293]:
scaling_pipline=Pipeline(
    steps=[
        ('num_scaling',RobustScaler())
    ]
)

In [294]:
encoding=Pipeline(
    steps=[
        ('encoding',OrdinalEncoder())
    ]
)

In [295]:
preprocessing=ColumnTransformer(
    transformers=[
        ('num_preprocess',scaling_pipline,num),
        #('cat_preprocess',encoding,cat)
    ]
)

In [296]:
main_pipeline=Pipeline(
    steps=[
        ("preprocessing",preprocessing),
        ('model',XGBClassifier(enable_categorical=True))
    ]
)

In [297]:
main_pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_preprocess',
                                                  Pipeline(steps=[('num_scaling',
                                                                   RobustScaler())]),
                                                  Index(['amount_aed', 'avg_item_price', 'ip_risk_score', 'card_age_days',
       'user_account_age_days', 'local_hour'],
      dtype='object'))])),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, c...
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

In [298]:
main_pipeline.score(x_train,y_train)

0.92095

In [299]:
main_pipeline.score(x_test,y_test)

0.9161